## Exploring information from student grades in 902, 10th, and 11th grade.

### 1. Loading data

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Machine Learning libraries
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.linear_model import LogisticRegression

# # MLflow for experiment tracking
# import mlflow
# import mlflow.sklearn
# from mlflow.models import infer_signature
# from mlflow.tracking import MlflowClient


## 2. Exploring data.

In [24]:
base = Path("./cleaned_data")
paths = sorted(base.glob("*.parquet"))

In [25]:
dfs = {p.stem : pd.read_parquet(p) for p in paths}

#### 2.1 Cleaning dataset

In [26]:
dfs["11_1_2"][21:26]

,Competencia,CONOCER,HACER,SER,CONVIVIR,Subtotal NIVEL,Nota P1,Rec P1,Nota P2,Rec P2,...,OBS 1,OBS 2,OBS 3,OBS 4,OBS 5,LLEGADA TARDE,INASISTENCIA JUSTIFICADA,INASISTENCIA INJUSTIFICADA,PERMISO,RETARDO
JOVEN POVEDA KHAROLL JULIANA,NaN,S,S,S,S,S,A,None,S,None,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
LEAL ECHEVERRY SANTIAGO,NaN,B,A,B,B,B,B,None,B,None,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
LOPEZ CHAVARRO KAREN ALEXANDRA,NaN,S,S,S,S,S,B,None,S,None,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
MARTINEZ CHAVES ISABELLA,NaN,S,S,S,S,S,S,None,S,None,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0
MATEUS VILLAMIL SAMUEL DAVID,NaN,A,A,B,B,A,B,None,A,None,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0


In [27]:
dfs["11_1_2"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, ACOSTA GUTIERREZ VERONICA SOFIA to VICTORIA RAMOS SARA SOFIA
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Competencia                 0 non-null      float64
 1   CONOCER                     40 non-null     object 
 2   HACER                       40 non-null     object 
 3   SER                         40 non-null     object 
 4   CONVIVIR                    40 non-null     object 
 5   Subtotal NIVEL              40 non-null     object 
 6   Nota P1                     41 non-null     object 
 7   Rec P1                      2 non-null      object 
 8   Nota P2                     40 non-null     object 
 9   Rec P2                      1 non-null      object 
 10  Nota P3                     0 non-null      float64
 11  Rec P3                      0 non-null      float64
 12  Nota PF                     41 non-null     ob

In [28]:
dfs["11_1_2"].isna().sum().sort_values(ascending=False)[:11].keys()

Index(['Competencia', 'OBS  4', 'OBS  5', 'OBS  2', 'Rec PF', 'OBS  1',
       'Nota P3', 'Rec P3', 'OBS  3', 'Rec P2', 'Rec P1'],
      dtype='object')

Columns to drop:

* `Competencia`
* `OBS n` where n = 1, 2, 3, 4, 5
* `Nota P3`,
* `Rec P3`,
* `Rec PF`

In [29]:
cols_to_drop = ['Competencia', 'OBS  4', 'OBS  5', 'OBS  2', 'Rec PF', 'OBS  1',
       'Nota P3', 'Rec P3', 'OBS  3']

In [30]:
test_df = dfs['11_1_2']
test_df = test_df.drop(columns=cols_to_drop)

In [31]:
test_df.dtypes

CONOCER                       object
HACER                         object
SER                           object
CONVIVIR                      object
Subtotal NIVEL                object
Nota P1                       object
Rec P1                        object
Nota P2                       object
Rec P2                        object
Nota PF                       object
LLEGADA TARDE                  int64
INASISTENCIA JUSTIFICADA       int64
INASISTENCIA INJUSTIFICADA     int64
PERMISO                        int64
RETARDO                        int64
dtype: object

In [32]:
test_df.select_dtypes(include="int64").value_counts() # No need to transform, passthrough

LLEGADA TARDE  INASISTENCIA JUSTIFICADA  INASISTENCIA INJUSTIFICADA  PERMISO  RETARDO
0              0                         0                           0        0          20
                                         1                           0        0           5
               1                         0                           0        0           3
1              0                         0                           0        0           3
0              0                         0                           1        0           2
                                         1                           2        0           1
               2                         0                           0        0           1
                                         2                           1        0           1
                                         1                           0        0           1
                                         3                           0        0       

In [33]:
test_df['Rec P1'].unique()

array([None, '3  B'], dtype=object)

In [34]:
# Columns to preprocess.
cat_cols = ['CONOCER', 'HACER', 'SER', 'CONVIVIR', 'Subtotal NIVEL']
rec_cols = ['Rec P1', 'Rec P2']
passthrough = test_df.select_dtypes(include="int64").columns.to_list()

In [43]:
rec_make_flags = FunctionTransformer(
    func=lambda X: X.notna().astype(int),
    feature_names_out="one-to-one"
)

In [44]:
# Pipelines.

# Categorical data pipeline
cat_pipe = Pipeline(
    [
        ("impute", SimpleImputer(strategy='constant', fill_value=1)), 
        ("ohe", OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ] 
)

# `Recuperacion` data pipeline
rec_pipe = Pipeline(
    [
        ("impute", rec_make_flags)
    ]
)

In [45]:
pre = ColumnTransformer(
    transformers=[
        ('cat', cat_pipe, cat_cols),
        ('rec', rec_pipe, rec_cols)
    ],
    remainder='passthrough',
    verbose_feature_names_out=True
)

In [46]:
X = test_df.drop(columns=["Nota P2"])
y = test_df[["Nota P2"]]

In [47]:
transformed = pre.fit_transform(X)

In [48]:
pre.set_params(verbose_feature_names_out=False)
pre.get_feature_names_out()

array(['CONOCER_A', 'CONOCER_B', 'CONOCER_S', 'CONOCER_None', 'HACER_A',
       'HACER_B', 'HACER_S', 'HACER_b', 'HACER_None', 'SER_A', 'SER_B',
       'SER_S', 'SER_None', 'CONVIVIR_A', 'CONVIVIR_B', 'CONVIVIR_S',
       'CONVIVIR_None', 'Subtotal NIVEL_A', 'Subtotal NIVEL_B',
       'Subtotal NIVEL_S', 'Subtotal NIVEL_b', 'Subtotal NIVEL_None',
       'Rec P1', 'Rec P2', 'Nota P1', 'Nota PF', 'LLEGADA TARDE',
       'INASISTENCIA JUSTIFICADA', 'INASISTENCIA INJUSTIFICADA',
       'PERMISO', 'RETARDO'], dtype=object)

In [49]:
modified_test_df = pd.DataFrame(
    data=transformed,
    index=test_df.index,
    columns=pre.get_feature_names_out()
)

In [55]:
print(f"Number of missing values is: {modified_test_df.isna().sum().sum()}")

Number of missing values is: 0


In [56]:
with open("new_file.csv", "w") as new:
    new.write(modified_test_df.to_csv())